### Imports

In [35]:
import numpy as np
import pandas as pd
import ace_tools_open as tools
from case_studies import *
import time
import matplotlib.pyplot as plt

In [36]:
#Save case study functions, their derivatives and hessians in lists
fs = [f1, f3, f4, f5]
dfs = [df1, df3, df4, df5]
Hfs = [Hf1, Hf3, Hf4, Hf5]
fnames = ["f1", "f3", "f4", "f5"]

In [37]:

def alg12(Q, b, delta, epsilon):
    def p(lambd):
        return -np.linalg.inv(Q + lambd * np.identity(Q.shape[0])).dot(b)
    
    lambda1 = np.linalg.eigvalsh(Q)[0]
    

    if lambda1 > 0 and np.linalg.norm(p(0)) <= delta:
        return p(0)
    
    l = max(0, -lambda1)
    u = max(0, -lambda1) + 1
    
    while np.linalg.norm(p(u)) > delta:
        l = u
        u = 2 * u
    
    while True:
        lambdaMark = 0.5 * (l + u)
        pMark = p(lambdaMark)
        
        if np.linalg.norm(pMark) < delta and abs(np.linalg.norm(pMark) - delta) < epsilon:
            return pMark
        
        if np.linalg.norm(pMark) > delta:
            l = lambdaMark
        else:
            u = lambdaMark




In [38]:
def generate_problem(n=5, Q_type='posdef', feasibility='feasible'):
    A = np.random.randn(n, n)
    Q = A.T @ A
    if Q_type == 'negdef':
        Q = -Q

    if feasibility == 'feasible':
        x_star = np.random.randn(n) * 0.5
        delta = np.linalg.norm(x_star) + 0.1
    elif feasibility == 'infeasible':
        x_star = np.random.randn(n) * 3.0
        delta = np.linalg.norm(x_star) * 0.7
    else:
        raise ValueError("feasibility must be 'feasible' or 'infeasible'")

    b = -Q @ x_star
    return Q, b, delta, x_star


In [39]:

def test_all_cases():
    results = []
    cases = [
        ('posdef', 'feasible'),
        ('posdef', 'infeasible'),
        ('negdef', 'feasible'),
        ('negdef', 'infeasible')
    ]

    for Q_type, feasibility in cases:
        Q, b, delta, x_star = generate_problem(n=5, Q_type=Q_type, feasibility=feasibility)
        x_sol = alg12(Q, b, delta, epsilon=1e-6)

        print(f"--- Q={Q_type}, feasibility={feasibility} ---")
        print(f"||x_star|| = {np.linalg.norm(x_star):.4f}")
        print(f"||x_sol||  = {np.linalg.norm(x_sol):.4f}")
        print(f"delta      = {delta:.4f}")
        if feasibility == 'feasible':
            print(f"Distance between x_star and x_sol: {np.linalg.norm(x_sol - x_star):.6f}")

        results.append((Q_type, feasibility, Q, b, delta, np.linalg.norm(x_star), np.linalg.norm(x_sol)))
    
    columns = ["Q_type", "Feasibility", "Q", "b", "delta", "||x_star||", "||x_sol||"]

    df_results = pd.DataFrame(results, columns=columns)
    df_display = df_results.drop(columns=["Q", "b"])

    return df_results, df_display


In [40]:
df_results, df_display = test_all_cases()
tools.display_dataframe_to_user(name="Benchmark Results", dataframe=df_display)

--- Q=posdef, feasibility=feasible ---
||x_star|| = 1.2132
||x_sol||  = 1.2132
delta      = 1.3132
Distance between x_star and x_sol: 0.000000
--- Q=posdef, feasibility=infeasible ---
||x_star|| = 4.4558
||x_sol||  = 3.1190
delta      = 3.1190
--- Q=negdef, feasibility=feasible ---
||x_star|| = 1.2908
||x_sol||  = 1.3908
delta      = 1.3908
Distance between x_star and x_sol: 2.534085
--- Q=negdef, feasibility=infeasible ---
||x_star|| = 8.1596
||x_sol||  = 5.7117
delta      = 5.7117
Benchmark Results
